# Are our significant DMGs the core enrichment genes in our identified enriched pathways?
We want to combine what we've found for statistically signficant differentially methylated genes and connect it to our identified enriched pathways so we can start to make sense of things biologically.

I have generated two csv files for both **phase 1 warm vs. control oysters**:
- phase1_wc_genes.csv - list of significant (adjusted p-value < 0.05) DMGs
- p1_wc_pathway.csv - list of enriched pathways from KEGG

Each enriched pathway contains a list of 'core enrichment genes' - these are a list of genes that are reported as part of the 'core enrichment' and contribute to the observed enrichment score.

The thinking is that maybe some of our signficant DMGs are part of that core enrichment group, which could tell us that that pathway is especially important/biologically relevant.

#### I. Load packages

In [3]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


#### II. Load, clean, and prep both csv files

In [2]:
# load in csv file
pathway <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_kegg_pathways/p1_wc_pathway.csv')

# clean headers and columns
pathway <- pathway[,-1]

# checking dimensions
dim(pathway) #119 pathways, 11 rows of info/meta data

head(pathway)

[1] 119  11

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
1,cvn00910,Nitrogen metabolism,10,0.7751841,1.619242,0.007876028,0.2056087,0.1873304,1398,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592
2,cvn00053,Ascorbate and aldarate metabolism,11,0.7413812,1.583931,0.013924094,0.2056087,0.1873304,2374,"tags=55%, list=18%, signal=45%",111124535/111103451/111124599/111112920/111115614/111103498
3,cvn00052,Galactose metabolism,21,0.6484812,1.582238,0.015374308,0.2056087,0.1873304,2261,"tags=38%, list=17%, signal=32%",111101197/111118471/111101820/111113388/111109442/111099882/111120703/111118006
4,cvn00592,alpha-Linolenic acid metabolism,11,0.7377666,1.576208,0.015550238,0.2056087,0.1873304,1602,"tags=45%, list=12%, signal=40%",111113990/111115744/111107112/111115745/111124908
5,cvn00511,Other glycan degradation,37,0.5917499,1.563906,0.004948574,0.2056087,0.1873304,2221,"tags=30%, list=16%, signal=25%",111106921/111106925/111106928/111119851/111119435/111120040/111113388/111119434/111106926/111119431/111106930
6,cvn03250,Viral life cycle,28,0.6023539,1.532503,0.008064620,0.2056087,0.1873304,2628,"tags=46%, list=19%, signal=37%",111124701/111124696/111129825/111111579/111108190/111135084/111128997/111124977/111106750/111123417/111130886/111104027/111135329


In [3]:
# load data frame
genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_sig_genes/phase1_wc_genes.csv')

# KEGG uses entrez IDs, which are my ensembl IDs without the 'LOC' in front of them, so need to convert those
genes$X <- substr(genes$X, start = 4, stop = nchar(genes$X))

# only grabbing the columns I care about
genes <- select(genes, X, log2FoldChange, padj)

# renaming columns to make more sense
colnames(genes) <- c('gene', 'lfc', 'padj')

# checking dimensions
dim(genes) # 189 sig DMGs

head(genes)

[1] 189   3

,gene,lfc,padj
,<chr>,<dbl>,<dbl>
1,111128103,1.644558,0.0307415459
2,111137770,2.655903,0.0178939261
3,111111295,2.360240,0.0137418025
4,111125391,2.205531,0.0207473298
5,111110197,1.863842,0.0209235911
6,111115675,2.781707,0.0000792069


#### III. Are the signficant genes in the core enrichment of the pathways?
Taking our list of siginificant genes, and going line by line in our pathways to see if our sig. gene matches any of the core enrichment genes

In [4]:
# generated from ChatGPT

# Sample dataframes
df1 <- genes
df2 <- pathway

# Function to check if any gene in df1 matches genes in a row of df2
get_gene_matches <- function(row_df2, df1_genes) {
  genes2 <- unlist(strsplit(as.character(row_df2), "/"))
  count <- sum(genes2 %in% df1_genes)
  return(count)
}

# Iterate over each row of df2
matches_count <- sapply(df2$core_enrichment, get_gene_matches, df1_genes = df1$gene)

# Add the matches count to df2
df2$MatchesCount <- matches_count

# Sort df with highest match counts at the top
gene_pathway_match <- df2[order(-df2$MatchesCount),]
head(gene_pathway_match)


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesCount
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
9,cvn04814,Motor proteins,110,0.4860525,1.448496,0.008328566,0.2056087,0.1873304,1179,"tags=18%, list=9%, signal=17%",111136151/111134768/111107338/111102596/111112439/111103394/111115784/111129526/111107250/111127380/111134843/111137068/111131563/111119946/111134888/111120500/111129376/111130940/111125250/111131555,7
15,cvn04144,Endocytosis,130,0.4435434,1.329267,0.030581040,0.2426096,0.2210419,3185,"tags=32%, list=24%, signal=25%",111120187/111125099/111112319/111119513/111112439/111119512/111136896/111104852/111102907/111107174/111136866/111125956/111134954/111112863/111116971/111112700/111135084/111133388/111134171/111121253/111115795/111127289/111129312/111134242/111121437/111129503/111135594/111106223/111123210/111125223/111105462/111104835/111104585/111104028/111133563/111105923/111119177/111101822/111104196/111137900/111112119/111123772,5
41,cvn03040,Spliceosome,91,0.4067843,1.195986,0.179752066,0.5397379,0.4917559,1557,"tags=16%, list=11%, signal=15%",111112733/111137770/111119513/111119512/111129112/111133954/111121854/111121021/111118318/111119442/111134531/111114893/111136440/111135640/111136164,4
1,cvn00910,Nitrogen metabolism,10,0.7751841,1.619242,0.007876028,0.2056087,0.1873304,1398,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592,3
30,cvn04070,Phosphatidylinositol signaling system,55,0.4596708,1.283800,0.115015974,0.5397379,0.4917559,2592,"tags=27%, list=19%, signal=22%",111101050/111127799/111125442/111100277/111125100/111126338/111100148/111135914/111134544/111120505/111135557/111122823/111128823/111138290/111130239,3
34,cvn00562,Inositol phosphate metabolism,46,0.4548720,1.242543,0.155361050,0.5397379,0.4917559,2592,"tags=26%, list=19%, signal=21%",111101050/111127799/111125442/111125100/111126338/111100148/111135914/111134544/111120505/111135557/111138290/111130239,3


now that I have a df with counts of number of matches between core enrichment genes and significant DMGs, want to only look at those with matches (filter out any pathways that did not contain sig. DMGs in their core enrichment)

In [5]:
# only want to look at pathways that have significant genes in their core enrichment
matched_pathways <- filter(gene_pathway_match, gene_pathway_match$MatchesCount != 0)

# checking dimensions to see how many pathways we have 
dim(matched_pathways) # 41 matches

# looking at df
head(matched_pathways)

[1] 41 12

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesCount
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
1,cvn04814,Motor proteins,110,0.4860525,1.448496,0.008328566,0.2056087,0.1873304,1179,"tags=18%, list=9%, signal=17%",111136151/111134768/111107338/111102596/111112439/111103394/111115784/111129526/111107250/111127380/111134843/111137068/111131563/111119946/111134888/111120500/111129376/111130940/111125250/111131555,7
2,cvn04144,Endocytosis,130,0.4435434,1.329267,0.030581040,0.2426096,0.2210419,3185,"tags=32%, list=24%, signal=25%",111120187/111125099/111112319/111119513/111112439/111119512/111136896/111104852/111102907/111107174/111136866/111125956/111134954/111112863/111116971/111112700/111135084/111133388/111134171/111121253/111115795/111127289/111129312/111134242/111121437/111129503/111135594/111106223/111123210/111125223/111105462/111104835/111104585/111104028/111133563/111105923/111119177/111101822/111104196/111137900/111112119/111123772,5
3,cvn03040,Spliceosome,91,0.4067843,1.195986,0.179752066,0.5397379,0.4917559,1557,"tags=16%, list=11%, signal=15%",111112733/111137770/111119513/111119512/111129112/111133954/111121854/111121021/111118318/111119442/111134531/111114893/111136440/111135640/111136164,4
4,cvn00910,Nitrogen metabolism,10,0.7751841,1.619242,0.007876028,0.2056087,0.1873304,1398,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592,3
5,cvn04070,Phosphatidylinositol signaling system,55,0.4596708,1.283800,0.115015974,0.5397379,0.4917559,2592,"tags=27%, list=19%, signal=22%",111101050/111127799/111125442/111100277/111125100/111126338/111100148/111135914/111134544/111120505/111135557/111122823/111128823/111138290/111130239,3
6,cvn00562,Inositol phosphate metabolism,46,0.4548720,1.242543,0.155361050,0.5397379,0.4917559,2592,"tags=26%, list=19%, signal=21%",111101050/111127799/111125442/111125100/111126338/111100148/111135914/111134544/111120505/111135557/111138290/111130239,3


In [6]:
mean(matched_pathways$MatchesCount)
median(matched_pathways$MatchesCount)
sd(matched_pathways$MatchesCount)

[1] 1.804878

[1] 1

[1] 1.308043

**Stats on Matched Counts**

25 pathways with only 1 match

148 pathways with 0 matches

- mean number of matches: 1.8
- median number of matches: 1
- standard deviation of matches: 1.31

**motor proteins** have the highest number of significant DMGs in its core enrichment with 7 genes

## Investigating adjusted p-values with NA values
We're trying to figure out *why* we get NA values so we can decide if we want to exclude or keep those genes in our pathway analysis

Starting with looking at our list of genes with stat info from DESeq (normal lfcShrink and lfcThreshold=0.5)

In [61]:
unfilter_genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_sig_genes/unfiltered_phase1_wc_genes.csv')
head(unfilter_genes)

,X,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,LOC111120752,0.9171026,0.5117345,0.7219483,0.01625394,0.9870318,NA
2,LOC111109452,1.3692365,1.0072548,0.7205748,0.70395856,0.4814586,NA
3,LOC111124802,160.7331187,0.4591660,0.1879969,0.00000000,1.0000000,1
4,LOC111101273,55.0404755,0.4055670,0.2258263,0.00000000,1.0000000,1
5,LOC111101250,77.7947775,-0.6027753,0.2031945,-0.50579750,0.6129988,1
6,LOC111101262,188.9039051,0.1674635,0.1795188,0.00000000,1.0000000,1


filtering df to only include genes with NA for adjusted p-value

In [62]:
genes_w_na <- unfilter_genes[is.na(unfilter_genes$padj),]
dim(genes_w_na) # 5,776 genes with NA for adjusted p-value

[1] 5776    7

looking more into the stats of those genes with NA for adjusted p-value, specifically looking at the baseMean (since this is what the DESeq documentation points at)
> baseMean - the average of the normalized count values, dividing by size factors, taken over all samples

In [66]:
mean(genes_w_na$baseMean) # average = 4.5 counts per sample
sd(genes_w_na$baseMean) # standard deviation = 22

[1] 4.522107

[1] 22.24341

loading in counts matrix that was generated with featureCounts to pull out the genes with NA for adjusted p-value

In [63]:
# loading in counts matrix
counts_matrix <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/featureCounts_meta.csv')
head(counts_matrix)

,X,BBB.WBO.B21,BBB.WBV.B70,BBO.BBO.B16,BBO.BBY.B27,BBO.WBO.B16,BBO.WBV.B64,BBR.BBB.B50,BBR.BBG.B38,BBR.BBY.B26,⋯,WBR.BBY.W25,WBV.WBO.W23,WBV.WBR.W12,WBY.BBV.W65,WBY.BBY.W30,WPB.BPG.G45,WPO.BPO.G16,WPO.BPY.G28,WPR.BPY.G25,WPV.BPR.G11
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,LOC111120752,2,2,0,0,2,0,0,0,1,⋯,0,0,0,0,1,0,0,1,1,0
2,LOC111109452,0,0,0,0,1,1,10,12,0,⋯,0,1,0,1,2,15,0,0,3,0
3,LOC111124802,597,453,408,244,502,434,187,336,311,⋯,122,120,57,289,202,168,134,86,252,9
4,LOC111101273,173,146,115,75,171,158,64,115,137,⋯,43,45,18,87,81,71,52,38,73,1
5,LOC111101250,253,255,257,118,327,269,115,141,184,⋯,91,96,24,254,109,92,49,5,101,2
6,LOC111101262,645,634,502,364,647,677,246,389,355,⋯,184,146,52,387,251,188,131,78,283,12


In [43]:
# setting genees as row names
rownames(counts_matrix) = counts_matrix$X

# removing gene column (since now are rownames)
counts_matrix2 <- counts_matrix[,-1]
head(counts_matrix2)

,BBB.WBO.B21,BBB.WBV.B70,BBO.BBO.B16,BBO.BBY.B27,BBO.WBO.B16,BBO.WBV.B64,BBR.BBB.B50,BBR.BBG.B38,BBR.BBY.B26,BBY.WBG.B42,⋯,WBR.BBY.W25,WBV.WBO.W23,WBV.WBR.W12,WBY.BBV.W65,WBY.BBY.W30,WPB.BPG.G45,WPO.BPO.G16,WPO.BPY.G28,WPR.BPY.G25,WPV.BPR.G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,2,2,0,0,2,0,0,0,1,0,⋯,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,0,0,0,1,1,10,12,0,2,⋯,0,1,0,1,2,15,0,0,3,0
LOC111124802,597,453,408,244,502,434,187,336,311,110,⋯,122,120,57,289,202,168,134,86,252,9
LOC111101273,173,146,115,75,171,158,64,115,137,45,⋯,43,45,18,87,81,71,52,38,73,1
LOC111101250,253,255,257,118,327,269,115,141,184,49,⋯,91,96,24,254,109,92,49,5,101,2
LOC111101262,645,634,502,364,647,677,246,389,355,108,⋯,184,146,52,387,251,188,131,78,283,12


In [44]:
# replace the '.' with '-'
cleaned_column_names2 <- gsub('\\.', "-", colnames(counts_matrix2))
head(cleaned_column_names2)

# now assigning to the columns
colnames(counts_matrix2) = cleaned_column_names2
head(counts_matrix2)

[1] "BBB-WBO-B21" "BBB-WBV-B70" "BBO-BBO-B16" "BBO-BBY-B27" "BBO-WBO-B16"
[6] "BBO-WBV-B64"

,BBB-WBO-B21,BBB-WBV-B70,BBO-BBO-B16,BBO-BBY-B27,BBO-WBO-B16,BBO-WBV-B64,BBR-BBB-B50,BBR-BBG-B38,BBR-BBY-B26,BBY-WBG-B42,⋯,WBR-BBY-W25,WBV-WBO-W23,WBV-WBR-W12,WBY-BBV-W65,WBY-BBY-W30,WPB-BPG-G45,WPO-BPO-G16,WPO-BPY-G28,WPR-BPY-G25,WPV-BPR-G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,2,2,0,0,2,0,0,0,1,0,⋯,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,0,0,0,1,1,10,12,0,2,⋯,0,1,0,1,2,15,0,0,3,0
LOC111124802,597,453,408,244,502,434,187,336,311,110,⋯,122,120,57,289,202,168,134,86,252,9
LOC111101273,173,146,115,75,171,158,64,115,137,45,⋯,43,45,18,87,81,71,52,38,73,1
LOC111101250,253,255,257,118,327,269,115,141,184,49,⋯,91,96,24,254,109,92,49,5,101,2
LOC111101262,645,634,502,364,647,677,246,389,355,108,⋯,184,146,52,387,251,188,131,78,283,12


In [45]:
# creating new df of counts matrix of genes with NA for padj
counts_na <- counts_matrix2[rownames(counts_matrix2) %in% genes_w_na$X,]
dim(counts_na) # 5,776 genes 
head(counts_na)

[1] 5776   30

,BBB-WBO-B21,BBB-WBV-B70,BBO-BBO-B16,BBO-BBY-B27,BBO-WBO-B16,BBO-WBV-B64,BBR-BBB-B50,BBR-BBG-B38,BBR-BBY-B26,BBY-WBG-B42,⋯,WBR-BBY-W25,WBV-WBO-W23,WBV-WBR-W12,WBY-BBV-W65,WBY-BBY-W30,WPB-BPG-G45,WPO-BPO-G16,WPO-BPY-G28,WPR-BPY-G25,WPV-BPR-G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,2,2,0,0,2,0,0,0,1,0,⋯,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,0,0,0,1,1,10,12,0,2,⋯,0,1,0,1,2,15,0,0,3,0
LOC111102393,73,54,36,12,48,48,30,49,31,14,⋯,20,9,10,24,17,36,14,3,26,41
LOC111117743,7,3,0,1,7,3,2,4,0,1,⋯,0,0,0,4,1,0,39,0,0,0
LOC111109809,0,0,7,6,0,0,5,7,8,0,⋯,1,3,1,2,4,1,4,1,0,2
LOC111135172,0,0,0,0,0,1,0,0,1,0,⋯,0,0,0,0,0,0,0,0,1,0


also need to load in meta data so that I can pull out the right columns aka samples that were warm or control for phase 1

In [46]:
meta <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/CV_CE18_meta.csv')
head(meta)

,sample_ID,unique_ID,T1_code,T2_code,ID_code,T1_oxygen,T1_temp,Tank1,T2_oxygen,T2_temp,Tank2,Phase1,Phase2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2018--BBB-WBO-B21-CV,BBB-WBO-B21,BBB,WBO,B21,hypoxic,ambient,B,normoxic,ambient,O,hypoxic,control
2,2018--BBB-WBV-B70-CV,BBB-WBV-B70,BBB,WBV,B70,hypoxic,ambient,B,normoxic,ambient,V,hypoxic,control
3,2018--BBO-BBO-B16-CV,BBO-BBO-B16,BBO,BBO,B16,hypoxic,ambient,O,hypoxic,ambient,O,hypoxic,hypoxic
4,2018--BBO-BBY-B27-CV,BBO-BBY-B27,BBO,BBY,B27,hypoxic,ambient,O,hypoxic,ambient,Y,hypoxic,hypoxic
5,2018--BBO-WBO-B16-CV,BBO-WBO-B16,BBO,WBO,B16,hypoxic,ambient,O,normoxic,ambient,O,hypoxic,control
6,2018--BBO-WBV-B64-CV,BBO-WBV-B64,BBO,WBV,B64,hypoxic,ambient,O,normoxic,ambient,V,hypoxic,control


pulling out only the samples that were either warm or control for phase 1 treatment (we're ignoring the effects of phase 2 for this analysis)

In [52]:
p1_wc_meta <- filter(meta, meta$Phase1 == 'warm' | meta$Phase1 == 'control')
dim(p1_wc_meta) # 15 total samples
head(p1_wc_meta)

[1] 15 13

,sample_ID,unique_ID,T1_code,T2_code,ID_code,T1_oxygen,T1_temp,Tank1,T2_oxygen,T2_temp,Tank2,Phase1,Phase2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2018--WBB-WBV-W69-CV,WBB-WBV-W69,WBB,WBV,W69,normoxic,ambient,B,normoxic,ambient,V,control,control
2,2018--WBG-BBB-W56-CV,WBG-BBB-W56,WBG,BBB,W56,normoxic,ambient,G,hypoxic,ambient,B,control,hypoxic
3,2018--WBG-WBG-W44-CV,WBG-WBG-W44,WBG,WBG,W44,normoxic,ambient,G,normoxic,ambient,G,control,control
4,2018--WBO-BBR-W03-CV,WBO-BBR-W03,WBO,BBR,W03,normoxic,ambient,O,hypoxic,ambient,R,control,hypoxic
5,2018--WBO-WBV-W64-CV,WBO-WBV-W64,WBO,WBV,W64,normoxic,ambient,O,normoxic,ambient,V,control,control
6,2018--WBR-BBY-W25-CV,WBR-BBY-W25,WBR,BBY,W25,normoxic,ambient,R,hypoxic,ambient,Y,control,hypoxic


In [56]:
p1_wc_meta

sample_ID,unique_ID,T1_code,T2_code,ID_code,T1_oxygen,T1_temp,Tank1,T2_oxygen,T2_temp,Tank2,Phase1,Phase2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2018--WBB-WBV-W69-CV,WBB-WBV-W69,WBB,WBV,W69,normoxic,ambient,B,normoxic,ambient,V,control,control
2018--WBG-BBB-W56-CV,WBG-BBB-W56,WBG,BBB,W56,normoxic,ambient,G,hypoxic,ambient,B,control,hypoxic
2018--WBG-WBG-W44-CV,WBG-WBG-W44,WBG,WBG,W44,normoxic,ambient,G,normoxic,ambient,G,control,control
2018--WBO-BBR-W03-CV,WBO-BBR-W03,WBO,BBR,W03,normoxic,ambient,O,hypoxic,ambient,R,control,hypoxic
2018--WBO-WBV-W64-CV,WBO-WBV-W64,WBO,WBV,W64,normoxic,ambient,O,normoxic,ambient,V,control,control
2018--WBR-BBY-W25-CV,WBR-BBY-W25,WBR,BBY,W25,normoxic,ambient,R,hypoxic,ambient,Y,control,hypoxic
2018--WBV-WBO-W23-CV,WBV-WBO-W23,WBV,WBO,W23,normoxic,ambient,V,normoxic,ambient,O,control,control
2018--WBV-WBR-W12-CV,WBV-WBR-W12,WBV,WBR,W12,normoxic,ambient,V,normoxic,ambient,R,control,control
2018--WBY-BBV-W65-CV,WBY-BBV-W65,WBY,BBV,W65,normoxic,ambient,Y,hypoxic,ambient,V,control,hypoxic


In [67]:
# only looking at samples with control or warm for phase 1 in the counts matrix
filtered_df <- counts_na[,colnames(counts_na) %in% p1_wc_meta$unique_ID]
dim(filtered_df) # still seeing 5,776 genes with NA for padj and info for 15 samples
head(filtered_df,20)

[1] 5776   15

,WBB-WBV-W69,WBG-BBB-W56,WBG-WBG-W44,WBO-BBR-W03,WBO-WBV-W64,WBR-BBY-W25,WBV-WBO-W23,WBV-WBR-W12,WBY-BBV-W65,WBY-BBY-W30,WPB-BPG-G45,WPO-BPO-G16,WPO-BPY-G28,WPR-BPY-G25,WPV-BPR-G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,3,1,0,0,1,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,1,0,0,7,0,1,0,1,2,15,0,0,3,0
LOC111102393,36,53,12,1,56,20,9,10,24,17,36,14,3,26,41
LOC111117743,521,1,1,22,0,0,0,0,4,1,0,39,0,0,0
LOC111109809,3,5,2,0,3,1,3,1,2,4,1,4,1,0,2
LOC111135172,1,1,11,0,0,0,0,0,0,0,0,0,0,1,0
LOC111121058,231,121,15,15,86,35,34,17,63,49,58,32,188,77,6
LOC111138070,7,0,0,1,0,0,0,6,17,0,4,3,0,1,1
LOC111108878,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0


all of the **W##** samples are **control** and all of the **G##** samples are **warm** for phase 1

looking at this df in excel to more easily look at why we might get NA for those genes

In [58]:
write.csv(filtered_df, '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/featureCounts_analysis/na_p1_wc_countsmatrix.csv')

# Unfiltered DESeq DF and Pathway
still looking at significantly differentially methylated genes for phase 1 warm vs. control, but now genes get assigned NA for adjusted p-values only when the gene has low counts (so this now includes outliers, instead of assigning outliers NA), therefore, different genes will be kept in the analysis for enriched pathways

In [1]:
# load in csv file
pathway <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_kegg_pathways/unfiltered_pathways_p1wc.csv')

# clean headers and columns
pathway <- pathway[,-1]

# checking dimensions
dim(pathway) #119 pathways, 11 rows of info/meta data

head(pathway)

[1] 121  11

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
1,cvn00053,Ascorbate and aldarate metabolism,12,0.7513575,1.660482,0.009027135,0.2014094,0.176967,1456,"tags=50%, list=11%, signal=45%",111124535/111103451/111124599/111127562/111112920/111115614
2,cvn00910,Nitrogen metabolism,10,0.7752533,1.652303,0.006203042,0.2014094,0.176967,1396,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592
3,cvn00052,Galactose metabolism,21,0.6484600,1.626358,0.013316325,0.2014094,0.176967,2261,"tags=38%, list=17%, signal=32%",111101197/111118471/111101820/111113388/111109442/111099882/111120703/111118006
4,cvn00592,alpha-Linolenic acid metabolism,11,0.7376869,1.605929,0.012687250,0.2014094,0.176967,1603,"tags=45%, list=12%, signal=40%",111113990/111115744/111107112/111115745/111124908
5,cvn00511,Other glycan degradation,37,0.5915432,1.589607,0.006132531,0.2014094,0.176967,2224,"tags=30%, list=16%, signal=25%",111106921/111106925/111106928/111119851/111119435/111120040/111113388/111119434/111106926/111119431/111106930
6,cvn00380,Tryptophan metabolism,31,0.5834860,1.534869,0.012337888,0.2014094,0.176967,2378,"tags=42%, list=18%, signal=35%",111127901/111100724/111103451/111134248/111133558/111112920/111115614/111121380/111125148/111109254/111130627/111108303/111103498


In [5]:
# load data frame
genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/unfiltered_lfc_p1wc.csv')

# KEGG uses entrez IDs, which are my ensembl IDs without the 'LOC' in front of them, so need to convert those
genes$X <- substr(genes$X, start = 4, stop = nchar(genes$X))

# only grabbing the columns I care about
genes <- select(genes, X, log2FoldChange, padj)

# renaming columns to make more sense
colnames(genes) <- c('gene', 'lfc', 'padj')

# only selecting genes that have padj < 0.05
genes <- filter(genes, genes$padj < 0.05)

# checking dimensions
dim(genes) # 189 sig DMGs

head(genes)

[1] 189   3

,gene,lfc,padj
,<chr>,<dbl>,<dbl>
1,111128103,1.644796,3.065827e-02
2,111137770,2.657125,1.784408e-02
3,111111295,2.361073,1.369104e-02
4,111125391,2.206213,2.068009e-02
5,111110197,1.864207,2.086317e-02
6,111115675,2.782320,7.896812e-05


#### III. Are the signficant genes in the core enrichment of the pathways?
Taking our list of siginificant genes, and going line by line in our pathways to see if our sig. gene matches any of the core enrichment genes

In [6]:
# generated from ChatGPT

# Sample dataframes
df1 <- genes
df2 <- pathway

# Function to check if any gene in df1 matches genes in a row of df2
get_gene_matches <- function(row_df2, df1_genes) {
  genes2 <- unlist(strsplit(as.character(row_df2), "/"))
  count <- sum(genes2 %in% df1_genes)
  return(count)
}

# Iterate over each row of df2
matches_count <- sapply(df2$core_enrichment, get_gene_matches, df1_genes = df1$gene)

# Add the matches count to df2
df2$MatchesCount <- matches_count

# Sort df with highest match counts at the top
gene_pathway_match <- df2[order(-df2$MatchesCount),]
head(gene_pathway_match)


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesCount
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
8,cvn04814,Motor proteins,111,0.4819493,1.433587,0.012391263,0.2014094,0.1769670,1180,"tags=18%, list=9%, signal=17%",111136151/111134768/111107338/111102596/111112439/111103394/111115784/111129526/111107250/111127380/111134843/111137068/111131563/111119946/111134888/111120500/111129376/111130940/111125250/111131555,7
14,cvn04144,Endocytosis,132,0.4453853,1.335542,0.028225806,0.2593783,0.2279009,3187,"tags=33%, list=24%, signal=25%",111120187/111125099/111112319/111119513/111112439/111119512/111136896/111104852/111102907/111107174/111136866/111125956/111134954/111112863/111116971/111112700/111135084/111133388/111134171/111121253/111115795/111127289/111129312/111134242/111121335/111121437/111129503/111135594/111106223/111123210/111125223/111105462/111104835/111104585/111104028/111133563/111105923/111119177/111101822/111104196/111137900/111112119/111123772,5
45,cvn03040,Spliceosome,92,0.3987387,1.166808,0.203893443,0.5574901,0.4898347,1558,"tags=16%, list=12%, signal=15%",111112733/111137770/111119513/111119512/111129112/111133954/111121854/111121021/111118318/111119442/111134531/111114893/111136440/111135640/111136164,4
2,cvn00910,Nitrogen metabolism,10,0.7752533,1.652303,0.006203042,0.2014094,0.1769670,1396,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592,3
26,cvn00562,Inositol phosphate metabolism,47,0.4705413,1.303162,0.095548317,0.4446672,0.3907036,2592,"tags=28%, list=19%, signal=22%",111101050/111127799/111125442/111125100/111127562/111126338/111100148/111135914/111134544/111120505/111135557/111138290/111130239,3
27,cvn04070,Phosphatidylinositol signaling system,55,0.4596492,1.292000,0.103266596,0.4627873,0.4066248,2592,"tags=27%, list=19%, signal=22%",111101050/111127799/111125442/111100277/111125100/111126338/111100148/111135914/111134544/111120505/111135557/111122823/111128823/111138290/111130239,3


now that I have a df with counts of number of matches between core enrichment genes and significant DMGs, want to only look at those with matches (filter out any pathways that did not contain sig. DMGs in their core enrichment)

In [7]:
# only want to look at pathways that have significant genes in their core enrichment
matched_pathways <- filter(gene_pathway_match, gene_pathway_match$MatchesCount != 0)

# checking dimensions to see how many pathways we have 
dim(matched_pathways) # 41 matches

# looking at df
head(matched_pathways)

[1] 42 12

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment,MatchesCount
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
1,cvn04814,Motor proteins,111,0.4819493,1.433587,0.012391263,0.2014094,0.1769670,1180,"tags=18%, list=9%, signal=17%",111136151/111134768/111107338/111102596/111112439/111103394/111115784/111129526/111107250/111127380/111134843/111137068/111131563/111119946/111134888/111120500/111129376/111130940/111125250/111131555,7
2,cvn04144,Endocytosis,132,0.4453853,1.335542,0.028225806,0.2593783,0.2279009,3187,"tags=33%, list=24%, signal=25%",111120187/111125099/111112319/111119513/111112439/111119512/111136896/111104852/111102907/111107174/111136866/111125956/111134954/111112863/111116971/111112700/111135084/111133388/111134171/111121253/111115795/111127289/111129312/111134242/111121335/111121437/111129503/111135594/111106223/111123210/111125223/111105462/111104835/111104585/111104028/111133563/111105923/111119177/111101822/111104196/111137900/111112119/111123772,5
3,cvn03040,Spliceosome,92,0.3987387,1.166808,0.203893443,0.5574901,0.4898347,1558,"tags=16%, list=12%, signal=15%",111112733/111137770/111119513/111119512/111129112/111133954/111121854/111121021/111118318/111119442/111134531/111114893/111136440/111135640/111136164,4
4,cvn00910,Nitrogen metabolism,10,0.7752533,1.652303,0.006203042,0.2014094,0.1769670,1396,"tags=50%, list=10%, signal=45%",111134700/111100398/111100399/111126492/111135592,3
5,cvn00562,Inositol phosphate metabolism,47,0.4705413,1.303162,0.095548317,0.4446672,0.3907036,2592,"tags=28%, list=19%, signal=22%",111101050/111127799/111125442/111125100/111127562/111126338/111100148/111135914/111134544/111120505/111135557/111138290/111130239,3
6,cvn04070,Phosphatidylinositol signaling system,55,0.4596492,1.292000,0.103266596,0.4627873,0.4066248,2592,"tags=27%, list=19%, signal=22%",111101050/111127799/111125442/111100277/111125100/111126338/111100148/111135914/111134544/111120505/111135557/111122823/111128823/111138290/111130239,3


In [8]:
mean(matched_pathways$MatchesCount)
median(matched_pathways$MatchesCount)
sd(matched_pathways$MatchesCount)

[1] 1.809524

[1] 1

[1] 1.292343

**Stats on Matched Counts**

25 pathways with only 1 match

148 pathways with 0 matches

- mean number of matches: 1.8
- median number of matches: 1
- standard deviation of matches: 1.31

**motor proteins** have the highest number of significant DMGs in its core enrichment with 7 genes

**these results are the *same* as without filtering the outlier counts**

In [9]:
unfilter_genes <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/lfc_sig_genes/unfiltered_phase1_wc_genes.csv')
head(unfilter_genes)

,X,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,LOC111120752,0.9171026,0.5117345,0.7219483,0.01625394,0.9870318,NA
2,LOC111109452,1.3692365,1.0072548,0.7205748,0.70395856,0.4814586,NA
3,LOC111124802,160.7331187,0.4591660,0.1879969,0.00000000,1.0000000,1
4,LOC111101273,55.0404755,0.4055670,0.2258263,0.00000000,1.0000000,1
5,LOC111101250,77.7947775,-0.6027753,0.2031945,-0.50579750,0.6129988,1
6,LOC111101262,188.9039051,0.1674635,0.1795188,0.00000000,1.0000000,1


filtering df to only include genes with NA for adjusted p-value

In [62]:
genes_w_na <- unfilter_genes[is.na(unfilter_genes$padj),]
dim(genes_w_na) # 5,776 genes with NA for adjusted p-value

[1] 5776    7

looking more into the stats of those genes with NA for adjusted p-value, specifically looking at the baseMean (since this is what the DESeq documentation points at)
> baseMean - the average of the normalized count values, dividing by size factors, taken over all samples

In [66]:
mean(genes_w_na$baseMean) # average = 4.5 counts per sample
sd(genes_w_na$baseMean) # standard deviation = 22

[1] 4.522107

[1] 22.24341

loading in counts matrix that was generated with featureCounts to pull out the genes with NA for adjusted p-value

In [63]:
# loading in counts matrix
counts_matrix <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/featureCounts_meta.csv')
head(counts_matrix)

,X,BBB.WBO.B21,BBB.WBV.B70,BBO.BBO.B16,BBO.BBY.B27,BBO.WBO.B16,BBO.WBV.B64,BBR.BBB.B50,BBR.BBG.B38,BBR.BBY.B26,⋯,WBR.BBY.W25,WBV.WBO.W23,WBV.WBR.W12,WBY.BBV.W65,WBY.BBY.W30,WPB.BPG.G45,WPO.BPO.G16,WPO.BPY.G28,WPR.BPY.G25,WPV.BPR.G11
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,LOC111120752,2,2,0,0,2,0,0,0,1,⋯,0,0,0,0,1,0,0,1,1,0
2,LOC111109452,0,0,0,0,1,1,10,12,0,⋯,0,1,0,1,2,15,0,0,3,0
3,LOC111124802,597,453,408,244,502,434,187,336,311,⋯,122,120,57,289,202,168,134,86,252,9
4,LOC111101273,173,146,115,75,171,158,64,115,137,⋯,43,45,18,87,81,71,52,38,73,1
5,LOC111101250,253,255,257,118,327,269,115,141,184,⋯,91,96,24,254,109,92,49,5,101,2
6,LOC111101262,645,634,502,364,647,677,246,389,355,⋯,184,146,52,387,251,188,131,78,283,12


In [43]:
# setting genees as row names
rownames(counts_matrix) = counts_matrix$X

# removing gene column (since now are rownames)
counts_matrix2 <- counts_matrix[,-1]
head(counts_matrix2)

,BBB.WBO.B21,BBB.WBV.B70,BBO.BBO.B16,BBO.BBY.B27,BBO.WBO.B16,BBO.WBV.B64,BBR.BBB.B50,BBR.BBG.B38,BBR.BBY.B26,BBY.WBG.B42,⋯,WBR.BBY.W25,WBV.WBO.W23,WBV.WBR.W12,WBY.BBV.W65,WBY.BBY.W30,WPB.BPG.G45,WPO.BPO.G16,WPO.BPY.G28,WPR.BPY.G25,WPV.BPR.G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,2,2,0,0,2,0,0,0,1,0,⋯,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,0,0,0,1,1,10,12,0,2,⋯,0,1,0,1,2,15,0,0,3,0
LOC111124802,597,453,408,244,502,434,187,336,311,110,⋯,122,120,57,289,202,168,134,86,252,9
LOC111101273,173,146,115,75,171,158,64,115,137,45,⋯,43,45,18,87,81,71,52,38,73,1
LOC111101250,253,255,257,118,327,269,115,141,184,49,⋯,91,96,24,254,109,92,49,5,101,2
LOC111101262,645,634,502,364,647,677,246,389,355,108,⋯,184,146,52,387,251,188,131,78,283,12


In [44]:
# replace the '.' with '-'
cleaned_column_names2 <- gsub('\\.', "-", colnames(counts_matrix2))
head(cleaned_column_names2)

# now assigning to the columns
colnames(counts_matrix2) = cleaned_column_names2
head(counts_matrix2)

[1] "BBB-WBO-B21" "BBB-WBV-B70" "BBO-BBO-B16" "BBO-BBY-B27" "BBO-WBO-B16"
[6] "BBO-WBV-B64"

,BBB-WBO-B21,BBB-WBV-B70,BBO-BBO-B16,BBO-BBY-B27,BBO-WBO-B16,BBO-WBV-B64,BBR-BBB-B50,BBR-BBG-B38,BBR-BBY-B26,BBY-WBG-B42,⋯,WBR-BBY-W25,WBV-WBO-W23,WBV-WBR-W12,WBY-BBV-W65,WBY-BBY-W30,WPB-BPG-G45,WPO-BPO-G16,WPO-BPY-G28,WPR-BPY-G25,WPV-BPR-G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,2,2,0,0,2,0,0,0,1,0,⋯,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,0,0,0,1,1,10,12,0,2,⋯,0,1,0,1,2,15,0,0,3,0
LOC111124802,597,453,408,244,502,434,187,336,311,110,⋯,122,120,57,289,202,168,134,86,252,9
LOC111101273,173,146,115,75,171,158,64,115,137,45,⋯,43,45,18,87,81,71,52,38,73,1
LOC111101250,253,255,257,118,327,269,115,141,184,49,⋯,91,96,24,254,109,92,49,5,101,2
LOC111101262,645,634,502,364,647,677,246,389,355,108,⋯,184,146,52,387,251,188,131,78,283,12


In [45]:
# creating new df of counts matrix of genes with NA for padj
counts_na <- counts_matrix2[rownames(counts_matrix2) %in% genes_w_na$X,]
dim(counts_na) # 5,776 genes 
head(counts_na)

[1] 5776   30

,BBB-WBO-B21,BBB-WBV-B70,BBO-BBO-B16,BBO-BBY-B27,BBO-WBO-B16,BBO-WBV-B64,BBR-BBB-B50,BBR-BBG-B38,BBR-BBY-B26,BBY-WBG-B42,⋯,WBR-BBY-W25,WBV-WBO-W23,WBV-WBR-W12,WBY-BBV-W65,WBY-BBY-W30,WPB-BPG-G45,WPO-BPO-G16,WPO-BPY-G28,WPR-BPY-G25,WPV-BPR-G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,2,2,0,0,2,0,0,0,1,0,⋯,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,0,0,0,1,1,10,12,0,2,⋯,0,1,0,1,2,15,0,0,3,0
LOC111102393,73,54,36,12,48,48,30,49,31,14,⋯,20,9,10,24,17,36,14,3,26,41
LOC111117743,7,3,0,1,7,3,2,4,0,1,⋯,0,0,0,4,1,0,39,0,0,0
LOC111109809,0,0,7,6,0,0,5,7,8,0,⋯,1,3,1,2,4,1,4,1,0,2
LOC111135172,0,0,0,0,0,1,0,0,1,0,⋯,0,0,0,0,0,0,0,0,1,0


also need to load in meta data so that I can pull out the right columns aka samples that were warm or control for phase 1

In [46]:
meta <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/counts_and_meta/CV_CE18_meta.csv')
head(meta)

,sample_ID,unique_ID,T1_code,T2_code,ID_code,T1_oxygen,T1_temp,Tank1,T2_oxygen,T2_temp,Tank2,Phase1,Phase2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2018--BBB-WBO-B21-CV,BBB-WBO-B21,BBB,WBO,B21,hypoxic,ambient,B,normoxic,ambient,O,hypoxic,control
2,2018--BBB-WBV-B70-CV,BBB-WBV-B70,BBB,WBV,B70,hypoxic,ambient,B,normoxic,ambient,V,hypoxic,control
3,2018--BBO-BBO-B16-CV,BBO-BBO-B16,BBO,BBO,B16,hypoxic,ambient,O,hypoxic,ambient,O,hypoxic,hypoxic
4,2018--BBO-BBY-B27-CV,BBO-BBY-B27,BBO,BBY,B27,hypoxic,ambient,O,hypoxic,ambient,Y,hypoxic,hypoxic
5,2018--BBO-WBO-B16-CV,BBO-WBO-B16,BBO,WBO,B16,hypoxic,ambient,O,normoxic,ambient,O,hypoxic,control
6,2018--BBO-WBV-B64-CV,BBO-WBV-B64,BBO,WBV,B64,hypoxic,ambient,O,normoxic,ambient,V,hypoxic,control


pulling out only the samples that were either warm or control for phase 1 treatment (we're ignoring the effects of phase 2 for this analysis)

In [52]:
p1_wc_meta <- filter(meta, meta$Phase1 == 'warm' | meta$Phase1 == 'control')
dim(p1_wc_meta) # 15 total samples
head(p1_wc_meta)

[1] 15 13

,sample_ID,unique_ID,T1_code,T2_code,ID_code,T1_oxygen,T1_temp,Tank1,T2_oxygen,T2_temp,Tank2,Phase1,Phase2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2018--WBB-WBV-W69-CV,WBB-WBV-W69,WBB,WBV,W69,normoxic,ambient,B,normoxic,ambient,V,control,control
2,2018--WBG-BBB-W56-CV,WBG-BBB-W56,WBG,BBB,W56,normoxic,ambient,G,hypoxic,ambient,B,control,hypoxic
3,2018--WBG-WBG-W44-CV,WBG-WBG-W44,WBG,WBG,W44,normoxic,ambient,G,normoxic,ambient,G,control,control
4,2018--WBO-BBR-W03-CV,WBO-BBR-W03,WBO,BBR,W03,normoxic,ambient,O,hypoxic,ambient,R,control,hypoxic
5,2018--WBO-WBV-W64-CV,WBO-WBV-W64,WBO,WBV,W64,normoxic,ambient,O,normoxic,ambient,V,control,control
6,2018--WBR-BBY-W25-CV,WBR-BBY-W25,WBR,BBY,W25,normoxic,ambient,R,hypoxic,ambient,Y,control,hypoxic


In [56]:
p1_wc_meta

sample_ID,unique_ID,T1_code,T2_code,ID_code,T1_oxygen,T1_temp,Tank1,T2_oxygen,T2_temp,Tank2,Phase1,Phase2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2018--WBB-WBV-W69-CV,WBB-WBV-W69,WBB,WBV,W69,normoxic,ambient,B,normoxic,ambient,V,control,control
2018--WBG-BBB-W56-CV,WBG-BBB-W56,WBG,BBB,W56,normoxic,ambient,G,hypoxic,ambient,B,control,hypoxic
2018--WBG-WBG-W44-CV,WBG-WBG-W44,WBG,WBG,W44,normoxic,ambient,G,normoxic,ambient,G,control,control
2018--WBO-BBR-W03-CV,WBO-BBR-W03,WBO,BBR,W03,normoxic,ambient,O,hypoxic,ambient,R,control,hypoxic
2018--WBO-WBV-W64-CV,WBO-WBV-W64,WBO,WBV,W64,normoxic,ambient,O,normoxic,ambient,V,control,control
2018--WBR-BBY-W25-CV,WBR-BBY-W25,WBR,BBY,W25,normoxic,ambient,R,hypoxic,ambient,Y,control,hypoxic
2018--WBV-WBO-W23-CV,WBV-WBO-W23,WBV,WBO,W23,normoxic,ambient,V,normoxic,ambient,O,control,control
2018--WBV-WBR-W12-CV,WBV-WBR-W12,WBV,WBR,W12,normoxic,ambient,V,normoxic,ambient,R,control,control
2018--WBY-BBV-W65-CV,WBY-BBV-W65,WBY,BBV,W65,normoxic,ambient,Y,hypoxic,ambient,V,control,hypoxic


In [67]:
# only looking at samples with control or warm for phase 1 in the counts matrix
filtered_df <- counts_na[,colnames(counts_na) %in% p1_wc_meta$unique_ID]
dim(filtered_df) # still seeing 5,776 genes with NA for padj and info for 15 samples
head(filtered_df,20)

[1] 5776   15

,WBB-WBV-W69,WBG-BBB-W56,WBG-WBG-W44,WBO-BBR-W03,WBO-WBV-W64,WBR-BBY-W25,WBV-WBO-W23,WBV-WBR-W12,WBY-BBV-W65,WBY-BBY-W30,WPB-BPG-G45,WPO-BPO-G16,WPO-BPY-G28,WPR-BPY-G25,WPV-BPR-G11
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
LOC111120752,3,1,0,0,1,0,0,0,0,1,0,0,1,1,0
LOC111109452,0,1,0,0,7,0,1,0,1,2,15,0,0,3,0
LOC111102393,36,53,12,1,56,20,9,10,24,17,36,14,3,26,41
LOC111117743,521,1,1,22,0,0,0,0,4,1,0,39,0,0,0
LOC111109809,3,5,2,0,3,1,3,1,2,4,1,4,1,0,2
LOC111135172,1,1,11,0,0,0,0,0,0,0,0,0,0,1,0
LOC111121058,231,121,15,15,86,35,34,17,63,49,58,32,188,77,6
LOC111138070,7,0,0,1,0,0,0,6,17,0,4,3,0,1,1
LOC111108878,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0


all of the **W##** samples are **control** and all of the **G##** samples are **warm** for phase 1

looking at this df in excel to more easily look at why we might get NA for those genes

In [58]:
write.csv(filtered_df, '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/featureCounts_analysis/na_p1_wc_countsmatrix.csv')